# Formatting the data

At this point the data is saved locally in `csv` format as `sample_trips.csv`, we need to grab that file and throw it through the wringer to format it into a GeoJSON store that can be consumed by `d3.js`.

In [2]:
import googlemaps
import json
import os
from polyline.codec import PolylineCodec
import geojson
import pandas as pd

First retrieve the data from the `csv` store.

In [11]:
sample_trips = pd.read_csv("sample_trips.csv", index_col=0)

Set up Google Maps API access. You need to have your Google Maps API credentials stored locally as `google_maps_api_key.json` using the following format:

> `{ "key": "..." }`

See the [Google Developer Console](https://console.developers.google.com/) for information getting your own API key!

In [13]:
def import_credentials(filename='google_maps_api_key.json'):
    if filename in [f for f in os.listdir('.') if os.path.isfile(f)]:
        data = json.load(open(filename))['key']
        return data
    else:
        raise IOError(
            'This API requires a Google Maps credentials token to work. Did you forget to define one?')
        
gmaps = googlemaps.Client(key=import_credentials())

The following loop handles pass the data through the Google Maps API and passing it to a `GeoJSON`-formatted `dict` that we can save and have `d3.js` consume. This is a complex process; here are the steps:

1. Call the Google Maps Directions API using the `googlemaps` module and parse out the [encoded polylines](https://developers.google.com/maps/documentation/utilities/polylinealgorithm).
2. Use the `polyline` module to dehash these into coordinates.
3. The Google Maps Directions API represents each trip as a series of subcomponents called "legs". For our purposes we want the entire coordinate list proper, so we have to flatten these coordinate sublists into a single list.
4. Package these coordinates into a unified GeoJSON `FeaturesList`, in which each set of coordinates is a single `Feature` consisting of `Point` entities (all represented as objects from the `geojson` module in-code). (??? still a WIP...)
5. Save this as `sample_trips.geojson`. Done!

In [14]:
req = gmaps.directions([40.76727216,-73.99392888], [40.701907,-74.013942], mode='bicycling')

In [15]:
polylines = [step['polyline']['points'] for step in [leg['steps'] for leg in req[0]['legs']][0]]
coords = []
for polyline in polylines:
    coords += PolylineCodec().decode(polyline)

In [16]:
coords

[(40.76723, -73.99396),
 (40.76713, -73.99372),
 (40.76713, -73.99372),
 (40.76651, -73.99418),
 (40.76651, -73.99418),
 (40.76737, -73.99622),
 (40.76747, -73.99644),
 (40.76747, -73.99644),
 (40.76729, -73.99658),
 (40.76684, -73.9969),
 (40.76684, -73.9969),
 (40.7669, -73.99703),
 (40.7669, -73.99703),
 (40.76564, -73.99793),
 (40.76555, -73.99799),
 (40.76451, -73.99875),
 (40.76448, -73.99877),
 (40.76445, -73.9988),
 (40.76444, -73.99882),
 (40.76441, -73.99887),
 (40.76438, -73.99891),
 (40.76436, -73.99893),
 (40.76432, -73.99895),
 (40.76379, -73.99934),
 (40.76372, -73.99939),
 (40.76366, -73.99944),
 (40.76359, -73.99949),
 (40.76355, -73.99953),
 (40.7635, -73.99957),
 (40.76345, -73.99962),
 (40.7634, -73.99968),
 (40.76335, -73.99974),
 (40.76331, -73.99981),
 (40.76327, -73.99987),
 (40.76322, -73.99995),
 (40.76317, -74.00003),
 (40.76298, -74.00031),
 (40.76281, -74.00056),
 (40.76277, -74.00062),
 (40.76272, -74.00067),
 (40.76267, -74.00073),
 (40.7625, -74.00098),


In [ ]:
geojson.Feature, geojson.Point, geojson.FeatureCollection

In [ ]:
trips = [geojson.Feature(geometry=Point((x_coord, y_coord))) for x_coord, y_coord in coords]
trip = geojson.FeatureCollection

# my_feature = Feature(geometry=Point((1.6432, -19.123)))

# >>> my_other_feature = Feature(geometry=Point((-80.234, -22.532)))

# >>> FeatureCollection([my_feature, my_other_feature])  # doctest: +ELLIPSIS